In [50]:
import io, os, sys, types
!{sys.executable} -m pip install prettytable

You should consider upgrading via the 'pip install --upgrade pip' command.


In [51]:
import IPython
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell
from prettytable import PrettyTable
from collections import OrderedDict

In [52]:
def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

        
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod
    


class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

In [53]:
def colorfy_str(msg, color):
    '''Add color to string

    Keyword arguments:
    msg -- string that need to be coloured
    color -- choosen color (red, blue, green, grey, cyan)'''

    colors = {'red': '\033[0;31m', 'blue': '\033[0;34m', 'green': '\033[0;32m', 'grey': '\033[0;90m',
              'cyan': '\033[0;96m', 'reset': '\033[0;0m'}
    options = ','.join(colors.keys())
    return colors.get(color, '\ncolorfy function:\nNo such color available\nChoose one from: ' + options +'\n') + msg + colors['reset']


In [54]:
sys.meta_path.append(NotebookFinder())

In [57]:
from QUIZ_py3 import q_ans

ImportError: cannot import name 'q_ans'

In [48]:
answers

{1: '', 2: '', 3: '', 4: '', 5: '', 6: '', 7: '', 8: '', 9: '', 10: ''}

In [47]:
quiz_key = {1: 'c',
           2: 'd',
           3: 'b',
           4: 'c',
           5: 'c',
           6: 'd',
           7: 'd',
           8: 'a',
           9: 'a',
           10: 'b',}

In [37]:
result = PrettyTable()
result.field_names = answers.keys()
result_color = OrderedDict()
correct = 0

for k in answers.keys():
    if answers[k] == quiz_key[k]:
        result_color[k] = colorfy_str(answers[k], 'green')
        correct += 1
    elif answers[k] == '':
        result_color[k] = colorfy_str('-', 'red')
    else:
        result_color[k] = colorfy_str(answers[k], 'red')

result.add_row(result_color.values())
print(colorfy_str('\nYour result: (%d/%d)' %(correct, len(quiz_key.keys())), 'blue'))
print(result)



Your result: (0/10)
+---+---+---+---+---+---+---+---+---+----+
| 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 |
+---+---+---+---+---+---+---+---+---+----+
| - | - | - | - | - | - | - | - | - | -  |
+---+---+---+---+---+---+---+---+---+----+
